In [1]:
import os
import time

#import itertools as it
import multiprocessing as mp
#import numpy as np
#import matplotlib.pyplot as plt

#from importlib import reload

locationPrefix = "/home/tfuser/programming"
homeLocation = os.getcwd()
libsLocation = os.path.join(locationPrefix, "gdrove", "tf")

os.chdir(libsLocation)
import tsp.tsp_proc as tsp
import all_aux as au
os.chdir(homeLocation)


print("Ok!")

FileNotFoundError: [Errno 2] No such file or directory: '/home/tfuser/programming/gdrove/tf'

In [ ]:
'''
13 towns, 14 procs: Ok! Total count: 239,500,800. Time taken: 106.48 sec.
14 towns, 14 procs: Ok! Total count: 3,113,510,400. Time taken: 1339.78 sec.
15 towns, 22 procs: Ok! Total count: 43,589,145,600. Time taken: 17702.65 sec.
'''

## INPUTS
numTowns = 13
headSize = 2
numWorkers = 16

print("Ok!")

In [ ]:
'''

'''

idlessTowns = tsp.generateTowns(n=numTowns)
townIds = list(range(len(idlessTowns)))
xs = [t[1][0] for t in idlessTowns]
ys = [t[1][1] for t in idlessTowns]

towns = [(i,)+town for i,town in zip(townIds, idlessTowns)]
tsp.plotTowns(towns)

In [ ]:
timeStart = time.time()

numRoutes = tsp.phuc(len(townIds)-1) // 2
print("HST: Expected usefull routes count: {}".format(numRoutes))

heads = tsp.headsFn(townIds, headSize=headSize)
heads = tsp.startEndShuffle(heads)
batches = tsp.batchList(heads, numBatches=numWorkers)

distMatrix = tsp.xyDistMatrix(xs, ys)

print("HST: creating workers")
quesToCnt = [mp.Queue() for _ in range(numWorkers)]
queToHost = mp.Queue()

workers = [mp.Process(target=tsp.cntWorkerFn(no, townIds, distMatrix, name="countFn_{}".format(no)),
                      args=(queToCnt, queToHost)) \
           for no,queToCnt in enumerate(quesToCnt)]

for worker in workers:
    worker.start()
    
for queToCnt,batch in zip(quesToCnt, batches):
    queToCnt.put(tsp.LoadMsg(load=batch))
    
numRunningWorkers = numWorkers
candidates = []
running = True
while running:
    msg = queToHost.get()
    
    if isinstance(msg, tsp.PoisonMsg):
        numRunningWorkers -= 1
        if numRunningWorkers < 1:
            running = False
            
    elif isinstance(msg, tsp.InfoMsg):
        print(msg.info)
        
    elif isinstance(msg, tsp.LoadMsg):
        candidates += [msg.load]
        
    else:
        print("HST: recieved a message of unknown purpose")
        
for queToCnt in quesToCnt:
    queToCnt.put(tsp.PoisonMsg())
    
for worker in workers:
    worker.join()
    
for que in quesToCnt:
    que.close()
        
minRoute = None
minDistance = 999.0
totalCount = 0
for c in candidates:
    if c[1] < minDistance:
        minRoute = c[0]
        minDistance = c[1]
    totalCount += c[2]
        
print("{} ==> {}".format(minRoute, minDistance))

timeTaken = time.time() - timeStart
print("Ok! Total count: {}. Time taken: {} sec.".format(totalCount, "%.2f"%timeTaken))

In [ ]:
targetRoute = [  [t for t in towns if t[0]==ms][0]  for ms in minRoute  ]
tsp.plotTowns(towns, shortestTravel=targetRoute, plotSize=10)

In [ ]:
"""
def headsFn(ids, headSize=1):
    ids = ids[1:]
    droppedNose = ids[-1]
    
    heads = list(it.permutations(ids, r=headSize))
    heads = [h for h in heads if h[0] != droppedNose]
    
    ##
    return(heads)


def rmTupleEnt(xs, x):
    if not isinstance(x, tuple):
        x = (x,)
        
    ##
    return(tuple(i for i in xs if i not in x))


def xyCountRoute(distMatrix, name=None):
    def underhood(route):
        runs = zip(route[:-1], route[1:])
        return(  sum([distMatrix[run] for run in runs])  )
    
    ##
    if name is not None:
        underhood.__name__ = name
    return(underhood)


def countHeadedRoutesFn(ids, distMatrix, infDistance=999.0, name=None):
    '''
    ids: List[Int], ordered id sequence, e.g. [0,1,2,3,4]
    '''
    def underhood(head):
        idRest = rmTupleEnt(ids[1:], head)
        
        gen = it.permutations(idRest)
        xyCountDistance = xyCountRoute(distMatrix)
        
        minRoute = None
        minDistance = infDistance
        numCount = 0
        
        running = True
        while running:
            try:
                routeVal = next(gen)
                route = (ids[0],) + head + routeVal + (ids[0],)
                
                if route[1] < route[-2]:
                    distance = xyCountDistance(route)

                    if distance < minDistance:
                        minRoute = route
                        minDistance = distance

                    numCount += 1
                
            except StopIteration:
                running = False       
        
        ##
        return(minRoute, minDistance, numCount)
    
    ##
    if name is not None:
        underhood.__name__ = name
    return(underhood)





##  ##  TEST
n = 11
headSize = 1

ids = list(range(n))
print(ids, end="\n\n")

## TEST 1: headsFn
if False:
    heads = headsFn(ids, headSize=headSize)
    print(heads, end="\n\n")
    
## TEST 2: rmTupleEnt
if False:
    heads = headsFn(ids, headSize=headSize)    
    head1 = heads[0]
    ret1 = rmTupleEnt(ids[1:], head1)
    print(head1, "==>", ret1)
    
    heads = headsFn(ids, headSize=2)    
    head2 = heads[-1]
    ret2 = rmTupleEnt(ids[1:], head2)
    print(head2, "==>", ret2)
    print()
    
## TEST 3: countHeadedRoutesFn
if False:
    townIds = list(range(11))
    headSize = 4
    xs = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
    ys = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
    distMatrix = tsp.xyDistMatrix(xs, ys)
    
    heads = headsFn(townIds, headSize=headSize)
    print("Number of {}-sized heads: {}".format(headSize, len(heads)))
    head4 = heads[4] 
    print("4-th head: {}".format(head4))
    idRest = rmTupleEnt(townIds[1:], head4)
    print("Combinations will be made with: {}".format(idRest))
    
    candidate = countHeadedRoutesFn(townIds, distMatrix)(head4)
    print(candidate)
    
"""
print("TBD")

In [2]:
'''
def headsFn2(ids, headSize=1):
    droppedNose = ids[-1]  
    heads = list(it.permutations(ids, r=headSize))
    heads = [h for h in heads if h[0] != ids[-1]]
    
    ##
    return(heads)


##  ##
ids = list(range(4))
cmb = list(it.permutations(ids))

if False:
    for c in cmb:
        if c[0] < c[-1]:
            print(c)
        else:
            print(" "*20 + str(c))
            
print(len(cmb), "vs.", len(cmb)//2, end="\n+++++++++++++++++++++\n")


heads = headsFn2(ids, 2)
print(heads)
count = 0
for head in heads:
    restIds = rmTupleEnt(ids, head)
    gen = it.permutations(restIds)
    
    running = True
    while running:
        try:
            route = head + next(gen)
            if route[0] < route[-1]:
                print(route)
                count += 1
            else:
                print(" "*20 + str(route))
        except StopIteration:
            running = False
            
print("\n", count)
'''
print("Nothing ot see here")

Nothing ot see here


In [3]:
"""
def cntWorkerFn(no, townIds, distMatrix, infDistance=999.0, name=None):
    def underhood(queIn, queOut):
        queOut.put(tsp.InfoMsg(info="CNT{}: starting...".format(no)))
        
        minRoute = None
        minDistance = infDistance
        procCount = 0
        
        countDistanceFn = countHeadedRoutesFn(ids=townIds, distMatrix=distMatrix,
                                              infDistance=infDistance, name="Count_Distance_Fn_{}".format(no))
        
        running = True
        while running:
            msg = queIn.get()
            
            if isinstance(msg, tsp.PoisonMsg):
                running = False
                
            elif isinstance(msg, tsp.LoadMsg):
                heads = msg.load   
                queOut.put(tsp.InfoMsg(info="CNT{}: heads recieved: {}...".format(no, heads[0])))
            
                for head in heads:
                    route, distance, numCount = countDistanceFn(head)

                    if distance < minDistance:
                        minRoute = route
                        minDistance = distance
                        
                    procCount += numCount
                        
                queOut.put(tsp.LoadMsg(load=(minRoute, minDistance, procCount)))
                queOut.put(tsp.InfoMsg(info="CNT{}: candidates sent. Shutting down...".format(no)))
                queOut.put(tsp.PoisonMsg())
                        
            else:
                queOut.put(tsp.InfoMsg(info="CNT{}: recieved a message of unknown purpose".format(no)))
        
        ##
        return()
    
    ##
    if name is not None:
        underhood.__name__ = name
    return(underhood)


print("Ok!")
"""

print("TBD")

TBD


In [4]:
"""
def startEndShuffle(xs):
    return(list(it.chain.from_iterable(zip(xs, reversed(xs))))[:len(xs)])

##  ##  TESTS
if False:
    for i in range(7,14):
        xs = list(range(i))
        xsShf = startEndShuffle(xs)
        print(xs, "==>", xsShf)
        
"""
print("TBD")

TBD


In [5]:
'''
os.chdir(libsLocation)
reload(tsp)
os.chdir(homeLocation)

timeStart = time.time()

townIds = list(range(12))
headSize = 2

numWorkers = 6
numRoutes = tsp.phuc(len(townIds)-1) // 2

heads = headsFn(townIds, headSize=headSize)
heads = startEndShuffle(heads)
batches = tsp.batchList(heads, numBatches=numWorkers)

xs = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
ys = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
distMatrix = tsp.xyDistMatrix(xs, ys)

print("HST: creating workers")
quesToCnt = [mp.Queue() for _ in range(numWorkers)]
queToHost = mp.Queue()

workers = [mp.Process(target=cntWorkerFn(no, townIds, distMatrix, name="countFn_{}".format(no)),
                      args=(queToCnt, queToHost)) \
           for no,queToCnt in enumerate(quesToCnt)]

for worker in workers:
    worker.start()
    
for queToCnt,batch in zip(quesToCnt, batches):
    queToCnt.put(tsp.LoadMsg(load=batch))
    
numRunningWorkers = numWorkers
candidates = []
running = True
while running:
    msg = queToHost.get()
    
    if isinstance(msg, tsp.PoisonMsg):
        numRunningWorkers -= 1
        if numRunningWorkers < 1:
            running = False
            
    elif isinstance(msg, tsp.InfoMsg):
        print(msg.info)
        
    elif isinstance(msg, tsp.LoadMsg):
        candidates += [msg.load]
        
    else:
        print("HST: recieved a message of unknown purpose")
        
for queToCnt in quesToCnt:
    queToCnt.put(tsp.PoisonMsg())
        
minRoute = None
minDistance = 999.0
for c in candidates:
    if c[1] < minDistance:
        minRoute = c[0]
        minDistance = c[1]
        
print("{} ==> {}".format(minRoute, minDistance))

timeTaken = time.time() - timeStart
print("Ok! Time taken: {} sec.".format("%.2f"%timeTaken))
'''
print("Nothing happens here")

Nothing happens here


In [6]:
"""
def generateTowns(n=10, minval=-1.0, maxval=1.0):
    townNames = ["Bordertown", "Columbia", "Rapture", "Yharnam", "Yahar'Gul (UV)", 
                 "Genty Town", "St. Gojiras",
                 "(the FC of) Newark", "Nite-City", 
                 "Starlink Shrine", "Edge Knot City", "Mountain Knot City", "Vault 13", "Hemwick",
                 "San-Chelyabinsk", "Neuevasyuki", "Houston (WGP)", 
                 "Kryzhopl", "Bender's (Hold)", "Battleground"
                 "Zion", 
                 "Kuldakhar", "Eastheaven", "New Gettisburg", "Wellington Wells", "Redgrave",
                 "Sliabh Luachra", "Leithrim Fancy"]
    
    points = np.random.uniform(size=(n,2), low=minval, high=maxval)
    points = [tuple(i) for i in points]
    
    towns = [(name, coords) for name,coords in zip(townNames[:n], points)]
    return(towns)



def exclTown(towns, town):
    if not isinstance(town, list):
        town = [town]
    ## 
    return([t for t in towns if t not in town])



def plotTowns(towns, shortestTravel=None, plotSize=10):
    '''
    towns: List[Tuple(no, name, Tuple(coordX, coordY))] - full unsorted list of towns
    shortestTravel: Tuple(  Tuple(no, name, Tuple(coordX, coordY))), int  ) - one of shortest travels found and its length
    '''
    fig, ax = plt.subplots(1,1, figsize=(plotSize, plotSize))
    for town in towns:
        otherTowns = exclTown(towns, town)
        for t in otherTowns:
            ax.plot([town[2][0], t[2][0]], [town[2][1], t[2][1]], c="lightgrey")
    for town in towns:       
        ax.scatter(town[2][0], town[2][1])
        ax.text(x=town[2][0]+0.005, y=town[2][1]+0.005, s="{}: {}".format(town[0],town[1]))
        
    if shortestTravel is not None:
        shortXs = [t[2][0] for t in shortestTravel]
        shortYs = [t[2][1] for t in shortestTravel]
        
        ax.plot(shortXs, shortYs, c="red")
        
    ax.grid(ls="--")



##  ##
idlessTowns = generateTowns(n=15)
townIds = list(range(len(idlessTowns)))
xs = [t[1][0] for t in idlessTowns]
ys = [t[1][1] for t in idlessTowns]
towns = [(i,)+town for i,town in zip(townIds, idlessTowns)]
plotTowns(towns)

"""
print("TBD")

TBD


In [7]:
"""
timeStart = time.time()

headSize = 2
numWorkers = 22
numRoutes = tsp.phuc(len(townIds)-1) // 2
print("HST: Expected usefull routes count: {}".format(numRoutes))

heads = headsFn(townIds, headSize=headSize)
heads = startEndShuffle(heads)
batches = tsp.batchList(heads, numBatches=numWorkers)

distMatrix = tsp.xyDistMatrix(xs, ys)

print("HST: creating workers")
quesToCnt = [mp.Queue() for _ in range(numWorkers)]
queToHost = mp.Queue()

workers = [mp.Process(target=cntWorkerFn(no, townIds, distMatrix, name="countFn_{}".format(no)),
                      args=(queToCnt, queToHost)) \
           for no,queToCnt in enumerate(quesToCnt)]

for worker in workers:
    worker.start()
    
for queToCnt,batch in zip(quesToCnt, batches):
    queToCnt.put(tsp.LoadMsg(load=batch))
    
numRunningWorkers = numWorkers
candidates = []
running = True
while running:
    msg = queToHost.get()
    
    if isinstance(msg, tsp.PoisonMsg):
        numRunningWorkers -= 1
        if numRunningWorkers < 1:
            running = False
            
    elif isinstance(msg, tsp.InfoMsg):
        print(msg.info)
        
    elif isinstance(msg, tsp.LoadMsg):
        candidates += [msg.load]
        
    else:
        print("HST: recieved a message of unknown purpose")
        
for queToCnt in quesToCnt:
    queToCnt.put(tsp.PoisonMsg())
    
for worker in workers:
    worker.join()
    
for que in quesToCnt:
    que.close()
        
minRoute = None
minDistance = 999.0
totalCount = 0
for c in candidates:
    if c[1] < minDistance:
        minRoute = c[0]
        minDistance = c[1]
    totalCount += c[2]
        
print("{} ==> {}".format(minRoute, minDistance))

timeTaken = time.time() - timeStart
print("Ok! Total count: {}. Time taken: {} sec.".format(totalCount, "%.2f"%timeTaken))
"""
print("It's now Ok!")

It's now Ok!


In [8]:
'''
13 towns, 14 procs: Ok! Total count: 239,500,800. Time taken: 106.48 sec.
14 towns, 14 procs: Ok! Total count: 3,113,510,400. Time taken: 1339.78 sec.
15 towns, 22 procs: Ok! Total count: 43,589,145,600. Time taken: 17702.65 sec.
'''
print("Only reports valued")

Only reports valued
